<a href="https://colab.research.google.com/github/nikita-0209/image-classification/blob/master/CIFAR-10%20Default%20ResNet101%20Size.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [0]:
from tensorflow.python.keras.layers import Dense, Flatten, MaxPooling2D
from tensorflow.python.keras import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator

import matplotlib.pyplot as plt
import matplotlib.image as img
%matplotlib inline

import numpy as np
import cv2

import os

import pandas as pd

import random

from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()

Using TensorFlow backend.


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
from sklearn.model_selection import train_test_split

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
from tensorflow.keras.applications.resnet import ResNet101

# Load Dataset

In [0]:
#Dataset
downloaded = drive.CreateFile({'id':"1J615QNajDV3Q89TnE8R-iIvj_5zhMo1J"})   # replace the id with id of file you want to access
downloaded.GetContentFile('train_images.npy')        # replace the file name with your file

In [0]:
#Dataset
downloaded = drive.CreateFile({'id':"1tCHOmcsg96D9xfwY1Lf2-xZPT1PEUePe"})   # replace the id with id of file you want to access
downloaded.GetContentFile('train_labels.csv')        # replace the file name with your file

In [0]:
#Dataset
downloaded = drive.CreateFile({'id':"1ZdU78f7JN-OUI3Gd0UCFMwbr9RnoHk6G"})   # replace the id with id of file you want to access
downloaded.GetContentFile('test_images.npy')        # replace the file name with your file

In [0]:
testX=np.load('test_images.npy')

In [0]:
trainX=np.load('train_images.npy')

In [0]:
trainY=pd.read_csv('train_labels.csv')

# Preprocess
Remark: Here the shape of images is still (32,32,3), ie, the CIFAR-10 default size, not the ResNet 101 size because creating a numpy array with large dimensions causes Colab to crash. The resizing was repeated on every batch of data in the Image Generator Function.

In [0]:
validationX=trainX[40000:, :]
trainX=trainX[:40000, :]

In [0]:
validationY=trainY.iloc[40000:, 1]
trainY=trainY.iloc[:40000, 1]

In [0]:
def preprocessY (lst):
  integer_encoded = label_encoder.fit_transform(np.array(lst))
  trainY=to_categorical(integer_encoded)
  return trainY

In [0]:
def preprocessX(data):
  size=data.shape[0]
  data_new=np.zeros((size, 32, 32,3 ))
  for i in range (size):
    data_new[i]=np.dstack((data[i][0], data[i][1], data[i][2]))
    i=i+1
  
  return data_new

In [0]:
trainX=preprocessX(trainX)
validationX=preprocessX(validationX)

In [0]:
trainY=preprocessY(trainY)
validationY=preprocessY(validationY)

# Base Model: ResNet101

In [0]:
base_model = ResNet101(include_top=False, input_shape=(200, 200, 3))

171450368/171446536 [==============================] - 3s 0us/step


In [0]:
#Freeze all Layers of ResNET
base_model.trainable = False

In [0]:
base_model.summary()

Model: "resnet101"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 200, 200, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 206, 206, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 100, 100, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 100, 100, 64) 256         conv1_conv[0][0]                 
__________________________________________________________________________________________

# Model

In [0]:
layer1=Flatten()
layer2=Dense(1024, activation='relu')
layer3=Dense(10, activation='softmax')

In [0]:
model = Sequential([
  base_model, layer1, layer2, layer3])

In [0]:
model.compile(loss=CategoricalCrossentropy(),
              optimizer=Adam(),
              metrics=['accuracy'])

In [0]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet101 (Model)            (None, 7, 7, 2048)        42658176  
_________________________________________________________________
flatten (Flatten)            (None, 100352)            0         
_________________________________________________________________
dense (Dense)                (None, 1024)              102761472 
_________________________________________________________________
dense_1 (Dense)              (None, 10)                10250     
Total params: 145,429,898
Trainable params: 102,771,722
Non-trainable params: 42,658,176
_________________________________________________________________


In [0]:
model.load_weights("/content/drive/My Drive/check_cifar")

# Checkpoints

In [0]:
# Include the epoch in the file name (uses `str.format`)

checkpoint_path = '/content/drive/My Drive/check_cifar'
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights every epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    period=1)


# Generator Function

In [0]:
def gen(features, labels, batch_size):
 # Create empty arrays to contain batch of features and labels#
 batch_features = np.zeros((batch_size, 32, 32, 3))
 batch_features_new=np.zeros((batch_size, 200, 200, 3))
 batch_labels = np.zeros((batch_size,3))
 while True:
   for i in range(batch_size):
     # choose random index in features
     index= random.choices(range(len(labels)),k=batch_size)
     batch_features = features[index]
     batch_labels = labels[index]
     for j in range(batch_size):
       batch_features_new[j]=cv2.resize(batch_features[j], (200, 200), interpolation = cv2.INTER_AREA)
   yield batch_features_new, batch_labels

In [0]:
batch_size=64

In [0]:
history=model.fit_generator(gen(trainX, trainY, batch_size), 
                    steps_per_epoch=trainY.shape[0]//batch_size, 
                    epochs=10, 
                    validation_data=gen(validationX, validationY, batch_size),
                    validation_steps=validationY.shape[0]//batch_size,
                    callbacks=[cp_callback]
                    )

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/10
625/625 [==============================] - ETA: 0s - loss: 0.0714 - accuracy: 0.9794
Epoch 00001: saving model to /content/drive/My Drive/check_cifar
625/625 [==============================] - 1315s 2s/step - loss: 0.0714 - accuracy: 0.9794 - val_loss: 1.4077 - val_accuracy: 0.8105
Epoch 2/10
275/625 [============>.................] - ETA: 10:02 - loss: 0.0708 - accuracy: 0.9802